## Create Scheme

In [1]:
!rm test.db

In [2]:
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Table, Column, Integer, String, Float, DateTime, Boolean, DATE
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship, backref
from sqlalchemy.sql import func

import datetime

Base = declarative_base()

In [3]:
class User(Base):
    __tablename__ = "user"
    
    id = Column(Integer(), primary_key=True)
    name = Column(String(20), nullable=False)
    
    def __str__(self):
        return self.name
    
    
class Posts(Base):
    __tablename__ = "posts"
    
    id = Column(Integer(), primary_key=True, unique=True)
    title = Column(String(255), nullable=False)
    body = Column(String(2048), nullable=False)
    created_time = Column(DateTime(), default=func.now())
    view_count = Column(Integer(), default=10)
    owner_user_id = Column(Integer(), ForeignKey("user.id"))
    
    owner_user = relationship("User", backref=backref("posts"))
    
    def __str__(self):
        return f'title: {self.title}, body: {self.body}, view_count:{self.view_count} created_time: {self.created_time}, owner_user_id:{self.owner_user_id}'
    

In [4]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///test.db', echo=True)

Base.metadata.create_all(engine)

2020-08-08 12:40:27,213 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2020-08-08 12:40:27,213 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 12:40:27,214 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2020-08-08 12:40:27,215 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 12:40:27,216 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("user")
2020-08-08 12:40:27,216 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 12:40:27,217 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("user")
2020-08-08 12:40:27,217 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 12:40:27,218 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("posts")
2020-08-08 12:40:27,219 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 12:40:27,219 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("posts")
2020-08-08 12:40:27,220 INFO sqlalchemy.engine.base.Engine ()
2020-08-08 12:40:27,221 INFO

# Insert

In [5]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [6]:
users = [
    User(id=1, name="권태형"),
    User(id=2, name="박소현"),
    User(id=3, name="박유정"),
    User(id=4, name="박진형"),    
]
session.bulk_save_objects(users)
session.commit()

2020-08-08 12:40:27,240 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 12:40:27,241 INFO sqlalchemy.engine.base.Engine INSERT INTO user (id, name) VALUES (?, ?)
2020-08-08 12:40:27,241 INFO sqlalchemy.engine.base.Engine ((1, '권태형'), (2, '박소현'), (3, '박유정'), (4, '박진형'))
2020-08-08 12:40:27,243 INFO sqlalchemy.engine.base.Engine COMMIT


In [7]:
posts = [
    Posts(
        id=i, 
        title="title {}".format(i), 
        body="body {}".format(i), 
        owner_user_id=1,
    ) for i in range(1, 50)
]

session.bulk_save_objects(posts)
session.commit()

2020-08-08 12:40:27,252 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 12:40:27,254 INFO sqlalchemy.engine.base.Engine INSERT INTO posts (id, title, body, created_time, view_count, owner_user_id) VALUES (?, ?, ?, CURRENT_TIMESTAMP, ?, ?)
2020-08-08 12:40:27,255 INFO sqlalchemy.engine.base.Engine ((1, 'title 1', 'body 1', 10, 1), (2, 'title 2', 'body 2', 10, 1), (3, 'title 3', 'body 3', 10, 1), (4, 'title 4', 'body 4', 10, 1), (5, 'title 5', 'body 5', 10, 1), (6, 'title 6', 'body 6', 10, 1), (7, 'title 7', 'body 7', 10, 1), (8, 'title 8', 'body 8', 10, 1)  ... displaying 10 of 49 total bound parameter sets ...  (48, 'title 48', 'body 48', 10, 1), (49, 'title 49', 'body 49', 10, 1))
2020-08-08 12:40:27,257 INFO sqlalchemy.engine.base.Engine COMMIT


# SELECT

In [8]:
users = session.query(User).all()
for ele in users:
    print(ele)

2020-08-08 12:40:27,266 INFO sqlalchemy.engine.base.Engine BEGIN (implicit)
2020-08-08 12:40:27,268 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name 
FROM user
2020-08-08 12:40:27,269 INFO sqlalchemy.engine.base.Engine ()
권태형
박소현
박유정
박진형


In [9]:
posts = session.query(Posts).all()
for ele in posts:
    print(ele)

2020-08-08 12:40:27,278 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.body AS posts_body, posts.created_time AS posts_created_time, posts.view_count AS posts_view_count, posts.owner_user_id AS posts_owner_user_id 
FROM posts
2020-08-08 12:40:27,279 INFO sqlalchemy.engine.base.Engine ()
title: title 1, body: body 1, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 2, body: body 2, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 3, body: body 3, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 4, body: body 4, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 5, body: body 5, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 6, body: body 6, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 7, body: body 7, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_i

In [10]:
from sqlalchemy import desc

for user in session.query(User.name).order_by(desc(User.name)):
    print(user)

2020-08-08 12:40:27,291 INFO sqlalchemy.engine.base.Engine SELECT user.name AS user_name 
FROM user ORDER BY user.name DESC
2020-08-08 12:40:27,293 INFO sqlalchemy.engine.base.Engine ()
('박진형',)
('박유정',)
('박소현',)
('권태형',)


In [11]:
user = session.query(User).filter(User.id == 1).first()
print(user)

2020-08-08 12:40:27,303 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, user.name AS user_name 
FROM user 
WHERE user.id = ?
 LIMIT ? OFFSET ?
2020-08-08 12:40:27,304 INFO sqlalchemy.engine.base.Engine (1, 1, 0)
권태형


In [12]:
# 3
query = session.query(Posts).order_by(desc(Posts.created_time))
for result in query.all():
    print(result)

2020-08-08 12:40:27,312 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.body AS posts_body, posts.created_time AS posts_created_time, posts.view_count AS posts_view_count, posts.owner_user_id AS posts_owner_user_id 
FROM posts ORDER BY posts.created_time DESC
2020-08-08 12:40:27,313 INFO sqlalchemy.engine.base.Engine ()
title: title 1, body: body 1, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 2, body: body 2, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 3, body: body 3, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 4, body: body 4, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 5, body: body 5, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 6, body: body 6, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 7, body: body 7, view_count:10 created_time: 

In [13]:
# 4
result = session.query(Posts).filter(Posts.view_count <= 5).all()
print(result)

2020-08-08 12:40:27,327 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.body AS posts_body, posts.created_time AS posts_created_time, posts.view_count AS posts_view_count, posts.owner_user_id AS posts_owner_user_id 
FROM posts 
WHERE posts.view_count <= ?
2020-08-08 12:40:27,328 INFO sqlalchemy.engine.base.Engine (5,)
[]


In [22]:
# 5
query = session.query(User.id, Posts.owner_user_id, Posts.created_time)
query = query.join(Posts)
results = query.filter(Posts.created_time).all()

for row in results:
    print(row)

2020-08-08 12:45:59,255 INFO sqlalchemy.engine.base.Engine SELECT user.id AS user_id, posts.owner_user_id AS posts_owner_user_id, posts.created_time AS posts_created_time 
FROM user JOIN posts ON user.id = posts.owner_user_id
2020-08-08 12:45:59,256 INFO sqlalchemy.engine.base.Engine ()
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2020, 8, 8, 3, 40, 27))
(1, 1, datetime.datetime(2

In [14]:
query = session.query(Posts).order_by(desc(Posts.created_time))
for result in query.all():
    print(result)

2020-08-08 12:40:27,336 INFO sqlalchemy.engine.base.Engine SELECT posts.id AS posts_id, posts.title AS posts_title, posts.body AS posts_body, posts.created_time AS posts_created_time, posts.view_count AS posts_view_count, posts.owner_user_id AS posts_owner_user_id 
FROM posts ORDER BY posts.created_time DESC
2020-08-08 12:40:27,337 INFO sqlalchemy.engine.base.Engine ()
title: title 1, body: body 1, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 2, body: body 2, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 3, body: body 3, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 4, body: body 4, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 5, body: body 5, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 6, body: body 6, view_count:10 created_time: 2020-08-08 03:40:27, owner_user_id:1
title: title 7, body: body 7, view_count:10 created_time: 